# Calibration - VectorScaling vs MS-ODIR, replications 1-3 (Supp. Table 21)

##### This notebook contains code needed for Table 21 in Supplemental Material

1. The models need to be retrained using seed 13 and 45 (train_nn.py)
2. The new models has to be tuned using MS-ODIR and VectorScaling (tune_cal_guo.py and tune_cal_odir.py)

Evaluating calibration methods on convolutional neural networks.

In [1]:
import numpy as np
import pandas as pd
from os.path import join
import pickle
from calibration.cal_methods import VectorScaling_NN
from utility.unpickle_probs import unpickle_probs

Using TensorFlow backend.


In [2]:
import sys
from os import path
import os
import glob

In [3]:
from IPython.display import display
import pandas as pd

## Get the best Lambda for each model

Path to logits and tuning

In [4]:
PATH = join('..', '..', 'logits')
files_10 = ('probs_resnet_wide32_c10_seed13_logits.p', 'probs_resnet_wide32_c10_seed45_logits.p',
            'probs_resnet152_SD_SVHN_seed13_logits.p', 'probs_resnet152_SD_SVHN_seed45_logits.p',
            "logits_densenet40_c10_seed13_logits.p", "logits_densenet40_c10_seed45_logits.p",
            "logits_resenet110_c10_seed13_logits.p", "logits_resenet110_c10_seed45_logits.p",
            "logits_resnet110_SD_c10_seed13_logits.p", "logits_resnet110_SD_c10_seed45_logits.p",
            "logits_lenet_c10_seed13_logits.p", "logits_lenet_c10_seed45_logits.p")

files_100 = ('probs_resnet_wide32_c100_seed13_logits.p', 'probs_resnet_wide32_c100_seed45_logits.p',
             'probs_densenet40_c100_seed13_logits.p', 'probs_densenet40_c100_seed45_logits.p',
             'logits_resenet110_c100_seed13_logits.p', 'logits_resenet110_c100_seed45_logits.p',
             'logits_resnet110_SD_c100_seed13_logits.p', 'logits_resnet110_SD_c100_seed45_logits.p',
             'logits_lenet_c100_seed13_logits.p', 'logits_lenet_c100_seed45_logits.p')

files = files_10 + files_100

PATH_DF = join("..", "..", "tunings_all", "tunings_ms_odir")  # This folder should contain MS-ODIR dataframes with tunings
PATH_MODELS = join("..", "..", "model_weights") # And this folder all the weights generated while tuning MS-ODIR

In [5]:
cols = ["Error_test", "ECE_test", "ECE2_test", "ECE_CW_test", "ECE_CW2_test", "ECE_FULL_test", "ECE_FULL2_test", "MCE_test", "MCE2_test", "Loss_test", "Brier_test"]
cols_ens = [col + "_ens" for col in cols]

def read_in_results2(path, ext = ".p"):
    
    file_path = join(path, "*" + ext)
    files = glob.glob(file_path)



    dfs_list = []

    for fname in files:
        with open(fname, "rb") as f:
            df = pickle.load(f) 
            df_ens = df[1].loc[:, cols]
            df_ens.columns = cols_ens
            df_merged = pd.concat([df[0], df_ens], axis=1)    
            dfs_list.append(df_merged)
            
    df_tuning = pd.concat(dfs_list, sort=False)
    
    return df_tuning.reset_index(drop=True)

## Keras best results

In [6]:
df_tunings = read_in_results2(PATH_DF)


In [7]:
df_tunings.head()

,Name,L2,mu,Error,ECE,ECE2,ECE_CW,ECE_CW2,ECE_FULL,ECE_FULL2,...,ECE_test_ens,ECE2_test_ens,ECE_CW_test_ens,ECE_CW2_test_ens,ECE_FULL_test_ens,ECE_FULL2_test_ens,MCE_test_ens,MCE2_test_ens,Loss_test_ens,Brier_test_ens
0,resnet_wide32_c100_seed13,0.0025,1000.0,32.32,0.219235,-1.0,0.594155,-1.0,-1.0,-1.0,...,0.132079,-1,0.360732,-1,-1,-1,0.246361,-1,1.612790,0.004364
1,resnet_wide32_c100_seed13,0.0050,1000.0,32.00,0.204688,-1.0,0.573624,-1.0,-1.0,-1.0,...,0.127981,-1,0.348108,-1,-1,-1,0.279808,-1,1.489844,0.004282
2,resnet_wide32_c100_seed13,0.0100,1000.0,30.84,0.170285,-1.0,0.540593,-1.0,-1.0,-1.0,...,0.113462,-1,0.317261,-1,-1,-1,0.213696,-1,1.317504,0.004159
3,resnet_wide32_c100_seed13,0.0250,1000.0,29.26,0.120042,-1.0,0.498688,-1.0,-1.0,-1.0,...,0.085442,-1,0.261119,-1,-1,-1,0.173943,-1,1.128983,0.003933
4,resnet_wide32_c100_seed13,0.0500,1000.0,28.62,0.090809,-1.0,0.468521,-1.0,-1.0,-1.0,...,0.065229,-1,0.227784,-1,-1,-1,0.128920,-1,1.036958,0.003778


In [8]:
def get_best_validation(df, method_name, param = "Loss", equal_mu = False):


    names = df.Name.unique()
    
    results = []

    for name in names:
        
        df_sub = df[df.Name == name]
        if equal_mu:  # Get only equal L2 and mu values
            df_sub = df_sub[df_sub.mu == df_sub.L2]

        best_lambda = df_sub.sort_values([param]).iloc[0]["L2"]
        #best_mu = df_sub.sort_values([param]).iloc[0]["mu"]

        results.append(df_sub.loc[df_sub.Name == name].sort_values([param]).iloc[0])
        
    df_temp = pd.DataFrame(results, columns=df_sub.columns)
    cols = df_temp.columns
    cols = list(cols[:1]) + ["Method"] + list(cols[3:]) + ["Opt_by", "L2", "mu"]
    cols.remove("Brier_std")
    cols.remove("Loss_std")
    
    df_temp = df_temp.assign(Opt_by = param).assign(Method = method_name)
    
    
    return df_temp[cols].sort_values(["Name"])

In [9]:
def get_opt_results(df_opt, measure = "Loss_test", opt_by = ["Loss", "Brier", "Error", "ECE"], equal_mu = False):
    

    res = []
    columns = []

    for opt in opt_by:
        df = get_best_validation(df_opt, param=opt, equal_mu = equal_mu).sort_values(["Name"])
        res.append(df[measure].values)
        columns.append("Dirichlet NN -" + opt + " Opt")

    return pd.DataFrame(np.array(res).T, columns=columns)

In [10]:
df_ms = get_best_validation(df_tunings, method_name="ms_odir", param="Loss", equal_mu=False).round(7) #.to_clipboard(index=False, header=False)

In [11]:
df_ms #[cols_ens]

,Name,Method,Error,ECE,ECE2,ECE_CW,ECE_CW2,ECE_FULL,ECE_FULL2,MCE,...,ECE_CW2_test_ens,ECE_FULL_test_ens,ECE_FULL2_test_ens,MCE_test_ens,MCE2_test_ens,Loss_test_ens,Brier_test_ens,Opt_by,L2,mu
1062,densenet40_c100_seed13,ms_odir,31.180000,0.043187,-1.0,0.468458,-1.0,-1.0,-1.0,0.160415,...,-1,-1,-1,0.037568,-1,1.054557,0.003991,Loss,2.500,100000.00000
1467,densenet40_c100_seed45,ms_odir,29.140000,0.030100,-1.0,0.456487,-1.0,-1.0,-1.0,0.179015,...,-1,-1,-1,0.036731,-1,1.069182,0.004015,Loss,2.500,0.01000
3139,densenet40_c10_seed13,ms_odir,6.880000,0.018503,-1.0,0.085980,-1.0,-1.0,-1.0,0.377839,...,-1,-1,-1,0.253180,-1,0.211861,0.010388,Loss,0.500,0.00100
3867,densenet40_c10_seed45,ms_odir,6.300000,0.019665,-1.0,0.090025,-1.0,-1.0,-1.0,0.381157,...,-1,-1,-1,0.086971,-1,0.213249,0.010387,Loss,250.000,0.00100
2032,lenet_c100_seed13,ms_odir,58.200000,0.038307,-1.0,0.572437,-1.0,-1.0,-1.0,0.113333,...,-1,-1,-1,0.070048,-1,2.200537,0.007084,Loss,0.500,0.01000
2167,lenet_c100_seed45,ms_odir,58.360000,0.038767,-1.0,0.552427,-1.0,-1.0,-1.0,0.119798,...,-1,-1,-1,0.061154,-1,2.278866,0.007194,Loss,0.500,0.01000
6955,lenet_c10_seed13,ms_odir,24.460000,0.031278,-1.0,0.159529,-1.0,-1.0,-1.0,0.159104,...,-1,-1,-1,0.207069,-1,0.692872,0.032873,Loss,0.500,0.00100
7387,lenet_c10_seed45,ms_odir,23.900000,0.027997,-1.0,0.164878,-1.0,-1.0,-1.0,0.418659,...,-1,-1,-1,0.088328,-1,0.679737,0.032313,Loss,0.500,0.00100
250,resenet110_c100_seed13,ms_odir,29.840000,0.039067,-1.0,0.465052,-1.0,-1.0,-1.0,0.136877,...,-1,-1,-1,0.084002,-1,1.082672,0.003978,Loss,0.500,0.01000
763,resenet110_c100_seed45,ms_odir,29.560000,0.043726,-1.0,0.468798,-1.0,-1.0,-1.0,0.132156,...,-1,-1,-1,0.049624,-1,1.106721,0.004032,Loss,0.500,0.01000


In [12]:
def read_in_file(path, file):
    
    with open(join(path, file), "rb") as f:
        return pickle.load(f)

In [13]:
def get_best_models(path_models, df_opt, skip = False):
    
    models_names = os.listdir(path_models)

    #all_weights = {}
    #all_params = {}
    
    results = {}

    for i, row in df_opt.iterrows():
        print(row.Name)
        
        name = row.Name + "_"  # Otherwise also finds c100 files with c10 files
        
        if skip and "c100" in name:
            print("skip")
            
        else:
            l2 = "l2=%f" % row.L2
            mu = "mu=%f" % row.mu

            file = [f for f in models_names if (name in f and l2 in f and mu in f and "mean" not in f)]
            
            if len(file) != 0:
                file = file[0]  # 
                weights, _, _ = read_in_file(path_models, file)
                #all_weights[name[:-1]] = weights
                #all_params[name[:-1]] = 
                
                results[name[:-1]] = (weights, (l2, mu))

        
    return results

In [14]:
results = get_best_models(PATH_MODELS, df_ms)

densenet40_c100_seed13
densenet40_c100_seed45
densenet40_c10_seed13
densenet40_c10_seed45
lenet_c100_seed13
lenet_c100_seed45
lenet_c10_seed13
lenet_c10_seed45
resenet110_c100_seed13
resenet110_c100_seed45
resenet110_c10_seed13
resenet110_c10_seed45
resnet110_SD_c100_seed13
resnet110_SD_c100_seed45
resnet110_SD_c10_seed13
resnet110_SD_c10_seed45
resnet152_SD_SVHN_seed13
resnet152_SD_SVHN_seed45
resnet_wide32_c10
resnet_wide32_c100_seed13
resnet_wide32_c100_seed45
resnet_wide32_c10_seed13
resnet_wide32_c10_seed45


In [15]:
with open("best_weights_ms_odir.p", "wb") as f:
    pickle.dump(results, f)

## Get all weights for MS-ODIR, MS-ODIR-zero and VecS

In [16]:
# MS-ODIR weights and params
dict_weights_params = results

dict_weights_ms = {}
dict_params_ms = {}

for name, (ws, p) in dict_weights_params.items():
    
    weights = []

    for w in ws:
        w = np.hstack([w[0].T, w[1].reshape(-1,1)])
        weights.append(w)
    
    dict_weights_ms[name] = np.array(weights)
    dict_params_ms[name] = p

### MS-ODIR

In [17]:
weights_MS_ODIR = dict_weights_ms

### MS-ODIR-zero

In [18]:
weights_MS_zero = {}

for key, ws in dict_weights_ms.items():
    
    ws_new = []
    
    for i, w in enumerate(ws):
        
        w1 = np.hstack([np.diag(np.diag(w)), np.array(w[:, -1]).reshape(-1, 1)])  # First np.diag to get diagonal values, second to make it into diagonal matrix
        ws_new.append(w1)
        
    weights_MS_zero[key] = np.array(ws_new)

### VecS weights

In [19]:
weights_vec = {}

for f in files:
    # Load in the data
    (logits_val, y_val), (logits_test, y_test) = unpickle_probs(join(PATH, f))
    name = "_".join(f.split("_")[1:-1])  # file_name
    print(name)   

    mod = VectorScaling_NN()
    mod.fit(logits_val, y_val)

    w = mod.model.get_weights()
    w = np.hstack([np.diag(w[0]), w[1].reshape(-1,1)])

    weights_vec[name] = w

resnet_wide32_c10_seed13
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
diagonal_layer_1 (DiagonalLa (None, 10)                20        
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
5000/5000 [==============================] - 0s - loss: 0.3325     
Epoch 2/500
5000/5000 [==============================] - 0s - loss: 0.3244     
Epoch 3/500
5000/5000 [==============================] - 0s - loss: 0.3178     
Epoch 4/500
5000/5000 [==============================] - 0s - loss: 0.3134     
Epoch 5/500
5000/5000 [==============================] - 0s - loss: 0.3099   

5000/5000 [==============================] - 0s - loss: 0.1666     
Epoch 187/500
5000/5000 [==============================] - 0s - loss: 0.1667     
Epoch 188/500
5000/5000 [==============================] - 0s - loss: 0.1667     
Epoch 189/500
5000/5000 [==============================] - 0s - loss: 0.1664     
Epoch 190/500
5000/5000 [==============================] - 0s - loss: 0.1663     
Epoch 191/500
5000/5000 [==============================] - 0s - loss: 0.1664     
Epoch 192/500
5000/5000 [==============================] - 0s - loss: 0.1664     
Epoch 193/500
5000/5000 [==============================] - 0s - loss: 0.1664     
Epoch 194/500
5000/5000 [==============================] - 0s - loss: 0.1664     
Epoch 195/500
5000/5000 [==============================] - 0s - loss: 0.1664     
Epoch 196/500
5000/5000 [==============================] - 0s - loss: 0.1664     
Epoch 197/500
5000/5000 [==============================] - 0s - loss: 0.1666     
Epoch 198/500
5000/5000 [=====

5000/5000 [==============================] - 0s - loss: 0.1653     
Epoch 123/500
5000/5000 [==============================] - 0s - loss: 0.1656     
Epoch 124/500
5000/5000 [==============================] - 0s - loss: 0.1654     
Epoch 125/500
5000/5000 [==============================] - 0s - loss: 0.1653     
Epoch 126/500
5000/5000 [==============================] - 0s - loss: 0.1653     
Epoch 127/500
5000/5000 [==============================] - 0s - loss: 0.1652     
Epoch 128/500
5000/5000 [==============================] - 0s - loss: 0.1652     
Epoch 129/500
5000/5000 [==============================] - 0s - loss: 0.1654     
Epoch 130/500
5000/5000 [==============================] - 0s - loss: 0.1653     
Epoch 131/500
5000/5000 [==============================] - 0s - loss: 0.1653     
Epoch 132/500
5000/5000 [==============================] - 0s - loss: 0.1653     
Epoch 133/500
5000/5000 [==============================] - 0s - loss: 0.1652     
Epoch 134/500
5000/5000 [=====

6000/6000 [==============================] - 0s - loss: 0.0860     - ETA: 0s - loss: 0.086
Epoch 82/500
6000/6000 [==============================] - 0s - loss: 0.0860     
Epoch 83/500
6000/6000 [==============================] - 0s - loss: 0.0860     
Epoch 84/500
6000/6000 [==============================] - 0s - loss: 0.0860     
Epoch 85/500
6000/6000 [==============================] - 0s - loss: 0.0860     
Epoch 86/500
6000/6000 [==============================] - 0s - loss: 0.0860     
Epoch 87/500
6000/6000 [==============================] - 0s - loss: 0.0859     
Epoch 88/500
6000/6000 [==============================] - 0s - loss: 0.0859     
Epoch 89/500
6000/6000 [==============================] - 0s - loss: 0.0859     - ETA: 0s - loss: 0.09
Epoch 90/500
6000/6000 [==============================] - 0s - loss: 0.0859     
Epoch 91/500
6000/6000 [==============================] - 0s - loss: 0.0859     
Epoch 92/500
6000/6000 [==============================] - 0s - loss: 0.0859  

6000/6000 [==============================] - 0s - loss: 0.1058     
Epoch 119/500
6000/6000 [==============================] - 0s - loss: 0.1058     
Epoch 120/500
6000/6000 [==============================] - 0s - loss: 0.1058     
Epoch 121/500
6000/6000 [==============================] - 0s - loss: 0.1057     
Epoch 122/500
6000/6000 [==============================] - 0s - loss: 0.1057     
Epoch 123/500
6000/6000 [==============================] - 0s - loss: 0.1058     
Epoch 124/500
6000/6000 [==============================] - 0s - loss: 0.1058     
Epoch 125/500
6000/6000 [==============================] - 0s - loss: 0.1057     
Epoch 126/500
6000/6000 [==============================] - 0s - loss: 0.1058     
Epoch 127/500
6000/6000 [==============================] - 0s - loss: 0.1058     
Epoch 128/500
6000/6000 [==============================] - 0s - loss: 0.1058     
Epoch 129/500
6000/6000 [==============================] - 0s - loss: 0.1058     
Epoch 130/500
6000/6000 [=====

...

5000/5000 [==============================] - 0s - loss: 2.2232     
Epoch 207/500
5000/5000 [==============================] - 0s - loss: 2.2232     
Epoch 208/500
5000/5000 [==============================] - 0s - loss: 2.2230     
Epoch 209/500
5000/5000 [==============================] - 0s - loss: 2.2230     
Epoch 210/500
5000/5000 [==============================] - 0s - loss: 2.2231     
Epoch 211/500
5000/5000 [==============================] - 0s - loss: 2.2230     
Epoch 212/500
5000/5000 [==============================] - 0s - loss: 2.2232     
Epoch 213/500
5000/5000 [==============================] - 0s - loss: 2.2229     
Epoch 214/500
5000/5000 [==============================] - 0s - loss: 2.2227     
Epoch 215/500
5000/5000 [==============================] - 0s - loss: 2.2228     
Epoch 216/500
5000/5000 [==============================] - 0s - loss: 2.2230     
Epoch 217/500
5000/5000 [==============================] - 0s - loss: 2.2229     
Epoch 218/500
5000/5000 [=====

5000/5000 [==============================] - 0s - loss: 2.2215     
Epoch 307/500
5000/5000 [==============================] - 0s - loss: 2.2216     
Epoch 308/500
5000/5000 [==============================] - 0s - loss: 2.2215     
Epoch 309/500
5000/5000 [==============================] - 0s - loss: 2.2213     
Epoch 310/500
5000/5000 [==============================] - 0s - loss: 2.2214     
Epoch 311/500
5000/5000 [==============================] - 0s - loss: 2.2214     
Epoch 312/500
5000/5000 [==============================] - 0s - loss: 2.2211     
Epoch 313/500
5000/5000 [==============================] - 0s - loss: 2.2211     
Epoch 314/500
5000/5000 [==============================] - 0s - loss: 2.2211     
Epoch 315/500
5000/5000 [==============================] - 0s - loss: 2.2211     
Epoch 316/500
5000/5000 [==============================] - 0s - loss: 2.2212     
Epoch 317/500
5000/5000 [==============================] - 0s - loss: 2.2213     
Epoch 318/500
5000/5000 [=====

5000/5000 [==============================] - ETA: 0s - loss: 2.207 - 0s - loss: 2.2158     
Epoch 48/500
5000/5000 [==============================] - 0s - loss: 2.2155     
Epoch 49/500
5000/5000 [==============================] - 0s - loss: 2.2151     
Epoch 50/500
5000/5000 [==============================] - 0s - loss: 2.2150     
Epoch 51/500
5000/5000 [==============================] - 0s - loss: 2.2143     
Epoch 52/500
5000/5000 [==============================] - 0s - loss: 2.2142     
Epoch 53/500
5000/5000 [==============================] - 0s - loss: 2.2136     
Epoch 54/500
5000/5000 [==============================] - 0s - loss: 2.2136     
Epoch 55/500
5000/5000 [==============================] - 0s - loss: 2.2129     
Epoch 56/500
5000/5000 [==============================] - 0s - loss: 2.2130     
Epoch 57/500
5000/5000 [==============================] - 0s - loss: 2.2123     
Epoch 58/500
5000/5000 [==============================] - 0s - loss: 2.2122     
Epoch 59/500
5000

5000/5000 [==============================] - 0s - loss: 2.2008     
Epoch 148/500
5000/5000 [==============================] - 0s - loss: 2.2007     
Epoch 149/500
5000/5000 [==============================] - 0s - loss: 2.2008     
Epoch 150/500
5000/5000 [==============================] - 0s - loss: 2.2006     
Epoch 151/500
5000/5000 [==============================] - 0s - loss: 2.2006     
Epoch 152/500
5000/5000 [==============================] - 0s - loss: 2.2006     
Epoch 153/500
5000/5000 [==============================] - 0s - loss: 2.2004     
Epoch 154/500
5000/5000 [==============================] - 0s - loss: 2.2003     
Epoch 155/500
5000/5000 [==============================] - 0s - loss: 2.2004     
Epoch 156/500
5000/5000 [==============================] - 0s - loss: 2.2006     
Epoch 157/500
5000/5000 [==============================] - 0s - loss: 2.2001     
Epoch 158/500
5000/5000 [==============================] - 0s - loss: 2.2002     
Epoch 159/500
5000/5000 [=====

5000/5000 [==============================] - 0s - loss: 2.1970     
Epoch 347/500
5000/5000 [==============================] - 0s - loss: 2.1970     
Epoch 348/500
5000/5000 [==============================] - 0s - loss: 2.1973     
Epoch 349/500
5000/5000 [==============================] - 0s - loss: 2.1972     
Epoch 350/500
5000/5000 [==============================] - 0s - loss: 2.1971     
Epoch 351/500
5000/5000 [==============================] - 0s - loss: 2.1973     
Epoch 352/500
5000/5000 [==============================] - 0s - loss: 2.1971     
Epoch 353/500
5000/5000 [==============================] - 0s - loss: 2.1970     


### Get preds and compare results

In [21]:
def get_preds_logits(x, w):
    return softmax(x @ w[:, :-1].T + w[:, -1])

def get_preds(x, w):
    return softmax(np.log(clip_for_log(x)) @ w[:, :-1].T + w[:, -1])

def clip_for_log(X):
    eps = np.finfo(float).eps
    return np.clip(X, eps, 1-eps)

def softmax(x):
    """
    Compute softmax values for each sets of scores in x.
    
    Parameters:
        x (numpy.ndarray): array containing m samples with n-dimensions (m,n)
    Returns:
        x_softmax (numpy.ndarray) softmaxed values for initial (m,n) array
    """
    e_x = np.exp(x - np.max(x))  # Subtract max so biggest is 0 to avoid numerical instability
    
    # Axis 0 if only one dimensional array
    axis = 0 if len(e_x.shape) == 1 else 1
    
    return e_x / e_x.sum(axis=axis, keepdims=1)

##### Based on the weights generate the new predictions for each method


In [24]:

methods = ["MS-ODIR", "MS-ODIR-zero", "VecS"]
methods_weights = {
    "VecS": weights_vec,
    "MS-ODIR": weights_MS_ODIR,
    "MS-ODIR-zero": weights_MS_zero
}

name_method_preds = {}

for file in files:

    # Load in the data
    FILE_PATH = join(PATH, file)
    (logits_val, y_val), (logits_test, y_test) = unpickle_probs(FILE_PATH)
    name = "_".join(file.split("_")[1:-1])  # file_name
    print(name)
    preds = {}
    

    for method, w in methods_weights.items():    
        
        if method in ["Uncal", "TempS", "VecS", "MS-ODIR", "MS-ODIR-zero"]:
            pred_fn = get_preds_logits
            input_val = logits_val
            input_test = logits_test

        else:  # Dir-L2, Dir-ODIR
            pred_fn = get_preds
            input_val = softmax(logits_val)
            input_test = softmax(logits_test)

        w = w[name]

        if len(w.shape) == 3:  # Means we have multiple weight matrices

            preds_temp = []

            for w_i in w:
                preds_temp.append(pred_fn(input_test, w_i))

            preds[method] = np.mean(preds_temp, axis=0)

        else:  # TempS, VecS
            preds[method] = pred_fn(input_test, w)
    
    name_method_preds[name] = preds

resnet_wide32_c10_seed13
resnet_wide32_c10_seed45
resnet152_SD_SVHN_seed13
resnet152_SD_SVHN_seed45
densenet40_c10_seed13
densenet40_c10_seed45
resenet110_c10_seed13
resenet110_c10_seed45
resnet110_SD_c10_seed13
resnet110_SD_c10_seed45
lenet_c10_seed13
lenet_c10_seed45
resnet_wide32_c100_seed13
resnet_wide32_c100_seed45
densenet40_c100_seed13
densenet40_c100_seed45
resenet110_c100_seed13
resenet110_c100_seed45
resnet110_SD_c100_seed13
resnet110_SD_c100_seed45
lenet_c100_seed13
lenet_c100_seed45


##### Use predictions to evaluate the models

In [25]:
from utility.evaluation import evaluate_slim

results = []
columns = ["name", "method", "error_rate", "ece", "ece_cw", "mce", "loss", "brier"]
for file in files:

    # Load in the data
    FILE_PATH = join(PATH, file)
    (logits_val, y_val), (logits_test, y_test) = unpickle_probs(FILE_PATH)
    name = "_".join(file.split("_")[1:-1])  # file_name
    print(name)

    #suitable_methods = ["Uncal", "TempS", "VecS", "MS-ODIR", "Dir-L2", "Dir-ODIR", "MS-zero"]

    method_eval = {}

    for method in methods:         
        
        preds = name_method_preds[name][method]
        
        results.append([name, method, *evaluate_slim(preds, y_test)])
        
        
df = pd.DataFrame(results, columns=columns)

resnet_wide32_c10_seed13
resnet_wide32_c10_seed45
resnet152_SD_SVHN_seed13
resnet152_SD_SVHN_seed45
densenet40_c10_seed13
densenet40_c10_seed45
resenet110_c10_seed13
resenet110_c10_seed45
resnet110_SD_c10_seed13
resnet110_SD_c10_seed45
lenet_c10_seed13
lenet_c10_seed45
resnet_wide32_c100_seed13
resnet_wide32_c100_seed45
densenet40_c100_seed13
densenet40_c100_seed45
resenet110_c100_seed13
resenet110_c100_seed45
resnet110_SD_c100_seed13
resnet110_SD_c100_seed45
lenet_c100_seed13
lenet_c100_seed45


In [26]:
df.pivot("name", "method")["loss"]

method,MS-ODIR,MS-ODIR-zero,VecS
name,,,
densenet40_c100_seed13,1.054557,1.071072,1.063661
densenet40_c100_seed45,1.069182,1.085593,1.077035
densenet40_c10_seed13,0.211861,0.213659,0.213160
densenet40_c10_seed45,0.213248,0.213266,0.213498
lenet_c100_seed13,2.200537,2.223598,2.215459
lenet_c100_seed45,2.278866,2.294854,2.280543
lenet_c10_seed13,0.692872,0.693350,0.693919
lenet_c10_seed45,0.679737,0.681273,0.679555
resenet110_c100_seed13,1.082672,1.111157,1.090656
